## Housekeeping
Let's first set up our environment. 

In [1]:
import os, sys
# sys.path.insert(1, os.path.realpath(os.path.pardir) + "/notebooks/randao/beaconrunner")
sys.path.append("../..")
sys.path.append("../../..")

import types
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

from cadCAD_tools.profiling.visualizations import visualize_substep_impact

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.renderers.default = "plotly_mimetype+notebook_connected"
import plotly.graph_objects as go

from experiments.utils import display_code

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

Let's import some of the beaconrunner logic

## How does the RANDAO game work?

In [2]:
from model.validators.LazyValidator import LazyValidator
import model.simulator as simulator
import model.network as network
import model.specs as specs

num_validators = 16
validators = [LazyValidator(validator_index=i) for i in range(num_validators)]

# Create a genesis state
(genesis_state, genesis_block) = simulator.get_genesis_state_block(validators, seed="let's play randao")

# Validators load the state
[v.load_state(genesis_state.copy(), genesis_block.copy()) for v in validators]

# We skip the genesis block
simulator.skip_genesis_block(validators)

For simplicity let's just assume a fully connected network, meaning that all validators are connected with each other.

In [3]:
set_a = network.NetworkSet(validators=list(range(10)))
net = network.Network(validators = validators, sets = list([set_a]))

In [4]:
print("Genesis time =", validators[0].store.genesis_time, "seconds")
print("Store time =", validators[0].store.time, "seconds")
print("Current slot =", validators[0].data.slot)

Genesis time = 1607428800 seconds
Store time = 1607428812 seconds
Current slot = 1


In [5]:
def get_current_epoch_proposers():
    return get_epoch_proposers(epochs_ahead=0)

def get_next_epoch_proposers():
    return get_epoch_proposers(epochs_ahead=1)

def get_epoch_proposers(epochs_ahead=0):
    validator = net.validators[0]
    current_slot = specs.get_current_slot(validator.store)

    current_head_root = validator.get_head()
    current_state = validator.store.block_states[current_head_root].copy()
    current_epoch = specs.get_current_epoch(current_state)
    start_slot = specs.compute_start_slot_at_epoch(current_epoch)
    start_state = current_state.copy() if start_slot == current_state.slot else \
        validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()
    
    current_epoch_proposers = []
    
    for slot in range(epochs_ahead * specs.SLOTS_PER_EPOCH + start_slot, start_slot + (epochs_ahead+1) * specs.SLOTS_PER_EPOCH):
        if slot < start_state.slot:
            continue
        if start_state.slot < slot:
            specs.process_slots(start_state, slot)
        current_epoch_proposers.append(specs.get_beacon_proposer_index(start_state))
    return current_epoch_proposers

In [6]:
genesis_proposer_expectation = ""
for i in range(10):
    genesis_proposer_expectation += "Proposer indices for epoch {}: {} \n".format(i, get_epoch_proposers(epochs_ahead=i))
print(genesis_proposer_expectation)

Proposer indices for epoch 0: [5, 7, 15, 4] 
Proposer indices for epoch 1: [6, 0, 5, 9] 
Proposer indices for epoch 2: [15, 2, 15, 6] 
Proposer indices for epoch 3: [4, 12, 2, 12] 
Proposer indices for epoch 4: [2, 15, 14, 4] 
Proposer indices for epoch 5: [15, 2, 7, 9] 
Proposer indices for epoch 6: [1, 14, 2, 1] 
Proposer indices for epoch 7: [3, 11, 5, 12] 
Proposer indices for epoch 8: [12, 15, 9, 1] 
Proposer indices for epoch 9: [11, 14, 9, 3] 



We now have a schedule for who is expected to propose when. But hold on, what about the parameters `MIN_SEED_LOOKAHEAD=1` and `MAX_SEED_LOOKAHEAD=4`?!? Good catch! 

The `MAX_SEED_LOOKAHEAD` is actually the minimum delay on validator activations and exits; it basically means that validators strategically activating and exiting can only affect the seed 4 epochs into the future, leaving a space of 3 epochs within which proposers can mix-in unknown info to scramble the seed and hence make stake grinding via activating or exiting validators non-viable.

A random seed is used to select all the committees and proposers for an epoch. Every epoch, the beacon chain accumulates randomness from proposers via the RANDAO and stores it. The seed for the current epoch is based on the RANDAO output from the epoch `MIN_SEED_LOOKUP + 1` ago. With `MIN_SEED_LOOKAHEAD` set to one, the effect is that we can know the seed for the current epoch and the next epoch, but not beyond (since the next-but-one epoch depends on randomness from the current epoch that hasn't been accumulated yet).

Let's see if this is true, i.e. "Don't trust, but verify!"

### Simulations

Let's first create some observer functions that extract relevant data at each timestep for us

In [7]:
import experiments.templates.randao.observers as honest_randao_observers
display_code(honest_randao_observers)

import numpy as np

import model.specs as specs
import model.eth2 as eth2

# Common observers

def current_slot(params, substep, state_history, previous_state, policy_input):
    return ("current_slot", previous_state["network"].validators[0].data.slot)

def current_epoch(params, substep, state_history, previous_state, policy_input):
    return ("current_epoch", previous_state["network"].validators[0].data.current_epoch)

def get_epoch_proposers(validator, epochs_ahead=0):
    current_slot = specs.get_current_slot(validator.store)
    current_head_root = validator.get_head()
    current_state = validator.store.block_states[current_head_root].copy()
    current_epoch = specs.get_current_epoch(current_state)
    start_slot = specs.compute_start_slot_at_epoch(current_epoch)
    start_state = current_state.copy() if start_slot == current_state.slot else \
        validator.store.block_states[specs.get_block_root(current_state, current_epoch)].copy()

    epoch_proposers = []

    for slot in range(epochs_ahead * specs.SLOTS_PER_EPOCH + start_slot, start_slot + (epochs_ahead+1) * specs.SLOTS_PER_EPOCH):
        if slot < start_state.slot:
            continue
        if start_state.slot < slot:
            specs.process_slots(start_state, slot)
        epoch_proposers.append(specs.get_beacon_proposer_index(start_state))
    return epoch_proposers

def get_current_proposer_indices(params, substep, state_history, previous_state, policy_input):
    current_epoch_proposers = get_epoch_proposers(validator=previous_state["network"].validators[0], epochs_ahead=0)
    return ("current_epoch_proposer_indices", current_epoch_proposers)

def get_next_proposer_indices(params, substep, state_history, previous_state, policy_input):
    next_epoch_proposers = get_epoch_proposers(validator=previous_state["network"].validators[0], epochs_ahead=1)
    return ("next_epoch_proposer_indices", next_epoch_proposers)

def get_plus_2_epoch_proposer_indices(params, substep, state_history, previous_state, policy_input):
    plus_2_epoch_proposers = get_epoch_proposers(validator=previous_state["network"].validators[0], epochs_ahead=2)
    return ("plus_2_epoch_proposer_indices", plus_2_epoch_proposers)

def balances(params, substep, state_history, previous_state, policy_input):
    validators = previous_state["network"].validators
    validator = validators[0]
    head = specs.get_head(validator.store)
    current_state = validator.store.block_states[head]
    current_epoch = specs.get_current_epoch(current_state)
    indices = [i for i, v in enumerate(validators)]
    balances = [b for i, b in enumerate(current_state.balances)]
    return ("balances", [np.round(eth2.gwei_to_eth(balance), 6) for balance in balances])

We then run the following experiments, composed with three simulations, one per scenario we are interested to check.

In [8]:
from experiments.run import run
import experiments.templates.randao.experiment as randao_experiment

display_code(randao_experiment)

import copy

from experiments.default_experiment import experiment

import model.parts.observers as observers
import experiments.templates.randao.observers as randao_observers

import model.simulator as simulator
from model.network import Network, NetworkSet
from model.validators.RANDAOValidator import RANDAOValidator

# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

scenarios = ["honest", "skip", "slashable"]
# scenarios = ["slashable"]

randao_observers = {
    "current_epoch": randao_observers.current_epoch,
    "current_slot": randao_observers.current_slot,
    "current_epoch_proposer_indices": randao_observers.get_current_proposer_indices,
    "next_epoch_proposer_indices": randao_observers.get_next_proposer_indices,
    "plus_2_epoch_proposer_indices": randao_observers.get_plus_2_epoch_proposer_indices,
    "balances": randao_observers.balances,
}

def create_initial_network(scenario):
    num_validators = 20

    set_a = NetworkSet(validators=list(range(num_validators)))
    network_sets = list([set_a])

    validators = [RANDAOValidator(validator_index=i, scenario=scenario) for i in range(num_validators)]

    # Create a genesis state
    (genesis_state, genesis_block) = simulator.get_genesis_state_block(validators, seed="let's play randao")

    # Validators load the state
    [v.load_state(genesis_state.copy(), genesis_block.copy()) for v in validators]

    # We skip the genesis block
    simulator.skip_genesis_block(validators)

    network = Network(validators=validators, sets=network_sets)
    return network

def create_scenario_model(scenario):
    new_model = copy.deepcopy(experiment.simulations[0].model)
    new_model.initial_state["network"] = create_initial_network(scenario)
    parameter_overrides = {
        "num_epochs": [5]
    }
    new_model.params.update(parameter_overrides)

    (observed_ic, observed_psubs) = observers.add_observers(
        new_model.initial_state,
        new_model.state_update_blocks,
        randao_observers
    )

    new_model.initial_state = observed_ic
    new_model.state_update_blocks = observed_psubs

    return new_model

def create_simulation(scenario):
    new_simulation = copy.deepcopy(experiment.simulations[0])
    new_simulation.model = create_scenario_model(scenario)
    new_simulation.timesteps = simulator.get_timesteps(new_simulation)
    return new_simulation

experiment.simulations = [create_simulation(scenario) for scenario in scenarios]

In [9]:
# Experiment execution
df, exceptions = run(randao_experiment.experiment)

2021-09-28 13:06:05,956 - root - INFO - Running experiment
2021-09-28 13:06:06,241 - root - INFO - Starting simulation 0 / run 0 / subset 0
18 proposing block for slot 1
15 proposing block for slot 2
16 proposing block for slot 3
16 proposing block for slot 4
4 proposing block for slot 5
14 proposing block for slot 6
11 proposing block for slot 7
1 proposing block for slot 8
6 proposing block for slot 9
timestep 100 of run 1
11 proposing block for slot 10
10 proposing block for slot 11
8 proposing block for slot 12
17 proposing block for slot 13
1 proposing block for slot 14
10 proposing block for slot 15
18 proposing block for slot 16
8 proposing block for slot 17
timestep 200 of run 1
2 proposing block for slot 18
3 proposing block for slot 19
16 proposing block for slot 20
2021-09-28 13:12:29,905 - root - INFO - Starting simulation 1 / run 0 / subset 0
18 proposing block for slot 1
15 skipping block for slot 2
15 skipping block for slot 2
16 proposing block for slot 3
16 proposing b

### Experiment A

In slot 31 of epoch 0, a block was proposed for slot 31, then forward the state to slot 0 of epoch 1, record the proposer.

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). So propose a block in slot `3` as per usual and then forward state to next epoch. 

In [10]:
df[(df.simulation == 0) & (df.timestep % 12 == 1) & (df.current_slot % 4 == 2)][
    ["simulation", "current_epoch", "current_slot", "current_epoch_proposer_indices", "next_epoch_proposer_indices", "plus_2_epoch_proposer_indices", "balances"]]

,simulation,current_epoch,current_slot,current_epoch_proposer_indices,next_epoch_proposer_indices,plus_2_epoch_proposer_indices,balances
13,0,0,2,"[1, 18, 15, 16]","[16, 4, 14, 11]","[12, 14, 1, 4]","[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32...."
61,0,1,6,"[16, 4, 14, 11]","[1, 6, 11, 10]","[9, 18, 14, 9]","[32.0, 32.0004, 32.0004, 32.0, 32.0016, 32.0, ..."
109,0,2,10,"[1, 6, 11, 10]","[8, 17, 1, 10]","[2, 8, 2, 18]","[32.00162, 32.00402, 32.00242, 31.9984, 32.003..."
157,0,3,14,"[8, 17, 1, 10]","[18, 8, 2, 3]","[5, 13, 17, 9]","[32.00378, 32.00818, 32.00498, 32.00056, 32.00..."
205,0,4,18,"[18, 8, 2, 3]","[16, 16, 9, 11]","[4, 1, 2, 6]","[32.00594, 32.01074, 32.00914, 32.00272, 32.00..."


### Experiment B

In slot 31 let the proposer skip her duties, then forward state to slot 0 of epoch 1, record the proposer.

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). So skip block proposal for slot `3`, otherwise proceed as usual.

Are the proposers different between experiments A and B?

In [15]:
df[(df.simulation == 1) & (df.timestep % 12 == 1) & (df.current_slot % 4 == 3)][
    ["simulation", "current_epoch", "current_slot", "current_epoch_proposer_indices", "next_epoch_proposer_indices", "plus_2_epoch_proposer_indices", "balances"]]

,simulation,current_epoch,current_slot,current_epoch_proposer_indices,next_epoch_proposer_indices,plus_2_epoch_proposer_indices,balances
266,1,0,3,"[1, 18, 15, 16]","[16, 4, 14, 11]","[12, 14, 1, 4]","[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32...."
314,1,1,7,"[16, 4, 14, 11]","[17, 19, 8, 12]","[16, 16, 3, 17]","[32.0, 32.0003, 32.0003, 32.0, 32.0016, 32.0, ..."
362,1,2,11,"[17, 19, 8, 12]","[10, 1, 8, 10]","[18, 19, 19, 13]","[32.00148, 32.0018, 32.00208, 31.9984, 32.0030..."
410,1,3,15,"[10, 1, 8, 10]","[19, 17, 8, 14]","[6, 11, 0, 0]","[32.0035, 32.0053, 32.0044, 32.00042, 32.00468..."
458,1,4,19,"[19, 17, 8, 14]","[13, 13, 3, 13]","[3, 10, 8, 1]","[32.00552, 32.00762, 32.00672, 32.00244, 32.00..."


### Experiment C

The block at slot 31 of epoch 0 has a slashing event. Forward the state to slot 0 of epoch 1. Is it still the same proposer?

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). Create a slashable event for block proposer of slot `3`. Then forward state to next epoch and record proposers. 

Can we confirm the intuition that the proposer in experiment C is different than in experiment A&B?

In [16]:
df[(df.simulation == 2) & (df.timestep % 12 == 1) & (df.current_slot % 4 == 3)][
    ["simulation", "current_epoch", "current_slot", "current_epoch_proposer_indices", "next_epoch_proposer_indices", "plus_2_epoch_proposer_indices", "balances"]]

,simulation,current_epoch,current_slot,current_epoch_proposer_indices,next_epoch_proposer_indices,plus_2_epoch_proposer_indices,balances
507,2,0,3,"[1, 18, 15, 16]","[16, 4, 14, 11]","[12, 14, 1, 4]","[32.0, 31.9999, 31.9999, 32.0, 32.0, 32.0, 32...."
555,2,1,7,"[16, 4, 14, 11]","[1, 6, 11, 10]","[9, 18, 14, 9]","[32.0, 32.0001, 32.0001, 32.0, 32.001591, 32.0..."
603,2,2,11,"[1, 6, 11, 10]","[8, 17, 1, 10]","[2, 8, 2, 18]","[32.001407, 32.003299, 32.001707, 31.998399, 3..."
651,2,3,15,"[8, 17, 1, 10]","[18, 8, 2, 3]","[18, 18, 11, 1]","[32.003252, 31.50688, 32.003751, 32.000243, 32..."
699,2,4,19,"[18, 8, 2, 3]","[0, 4, 2, 11]","[7, 2, 4, 6]","[32.004992, 31.505476, 31.507219, 32.066088, 3..."


## Plotting the block tree

Let's visualize how the blockchain evolved

In [13]:
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz
from networkx.drawing.nx_agraph import pygraphviz_layout

2021-09-28 13:25:32,504 - matplotlib - DEBUG - matplotlib data path: /Users/barnabe/Documents/Research/Projects/beaconrunner/venv/lib/python3.8/site-packages/matplotlib/mpl-data
2021-09-28 13:25:32,514 - matplotlib - DEBUG - CONFIGDIR=/Users/barnabe/.matplotlib
2021-09-28 13:25:32,517 - matplotlib - DEBUG - matplotlib version 3.4.2
2021-09-28 13:25:32,518 - matplotlib - DEBUG - interactive is False
2021-09-28 13:25:32,518 - matplotlib - DEBUG - platform is darwin
2021-09-28 13:25:32,519 - matplotlib - DEBUG - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importli

In [14]:
# Just grab any Network() of any validator from the typical beaconrruner df output: 
# This allows us to access the validator'S store and recover block tree from it 
network = df["network"].iloc[0]

blocks_dict = network.validators[0].store.blocks # access blocks from validator's Store

block_headers = list(blocks_dict.values()) # extract block headers
block_roots = list(blocks_dict) # extract block roots

tree = nx.DiGraph()

# Add all blocks as nodes to directed graph
for index, block_header in enumerate(block_headers):
    tree.add_nodes_from([
    (index, {"slot": block_header.slot, "parent_root": block_header.parent_root, "block_root": block_roots[index]})
    ])

# Add edges between blocks respectively
for i in range(len(tree)):
    for j in range(len(tree)):
        if tree.nodes()[i]["parent_root"] == tree.nodes()[j]["block_root"]:
            tree.add_edge(i, j)

KeyError: 'network'

In [ ]:
# latest_block_root = network.validators[0].get_head()
# last_block_index = [x for x,y in tree.nodes(data=True) if y['block_root']==latest_block_root][0]

def get_node_index(block_root):
    return [x for x,y in tree.nodes(data=True) if y['block_root']==block_root][0]

def get_parent_node_index(block_index):
    parent_block_root = block_headers[block_index].parent_root
    parent_node_index = get_node_index(parent_block_root)
    return parent_node_index

def tag_chain(network):
    latest_block_root = network.validators[0].get_head()
    last_block_index = [x for x,y in tree.nodes(data=True) if y['block_root']==latest_block_root][0]

    canonical_chain = [last_block_index]
    query_next = last_block_index
    while True:
        parent = get_parent_node_index(query_next)
        canonical_chain.insert(0, parent)
        if parent == 0: #genesis
            break
        query_next = parent

    orphaned_chain = list(set(list(range(canonical_chain[-1]+1))) - set(canonical_chain))

    return canonical_chain, orphaned_chain

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4), sharex=True, sharey=True)

pos = pygraphviz_layout(tree, prog='dot', args='-Grankdir="RL"')
for i in pos:
    pos[i] = (block_headers[i].slot + 200, pos[i][1])

plt.title("Tree of blocks")

canonical_chain_ids, orphaned_chain_ids = tag_chain(network)

nx.draw_networkx_nodes(tree,pos=pos, nodelist=canonical_chain_ids, node_color='green', label="Canonical blocks", node_shape='s', node_size=500)
nx.draw_networkx_nodes(tree,pos=pos, nodelist=orphaned_chain_ids, node_color='orange', label="Orphaned blocks", node_shape='s', node_size=500)
nx.draw_networkx_nodes(tree,pos=pos, nodelist=[0], node_color='blue', label="Genesis block", node_shape='s', node_size=500)    

nx.draw_networkx_edges(tree, pos, arrows=True)
nx.draw_networkx_labels(tree, pos, font_size=10, font_color="white")

ax.set_frame_on(False)
plt.legend(loc="lower right", labelspacing=2, fontsize=9, frameon=False, borderpad=0.1)

# plt.legend(labelspacing=0.8, fontsize=9, frameon=False, borderpad=0.1)
plt.show()